In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor, Lambda
from dataclasses import dataclass

In [ ]:
import numpy as np
import pandas as pd
from rolling_and_plot import *

# %reload_ext autoreload
# %autoreload 2R

In [ ]:
# Get cpu or gpu device for training.
device = torch.device("cuda") #"mps" for M1 macs
print(f"Using {device} device")

# TOC

* [Data Loading](#dload)

* [Models](#model)

In [ ]:
@dataclass
class G:
#     split_time = None #for now, might be useless
    # sequence_length = 5
    num_features = 4 # delta_t, current, voltage, and soc at t-1
    window_size = 600
    batch_size = 16
    epochs = 5

# Data Loader <a id="dload"></a>

In [ ]:
# from google.colab import files
file = pd.read_csv("/content/JMFM_17_SOC_OCV_94plus5_Test_220624_soc.csv")

In [ ]:
data_plot(data = [file],
          title="OCV v SOC",
          x = ["test time (sec)"],
          y = ["soc"],
          markers = "lines"
         )

In [ ]:
train = file[["delta t","current","voltage","soc" ]].loc[:107898]

test = file[["delta t","current","voltage","soc" ]].loc[107899:]

In [ ]:
# np.array_split(train,len(train)//6) not applicable here

In [ ]:
train = rolling(train, G.window_size)
test = rolling(test, G.window_size)

In [ ]:
class BatterySet(Dataset):
    def __init__(self, dataframe):
            
        x, y = dataframe
    
        self.logits = torch.from_numpy(x).to(device)
        self.labels = torch.from_numpy(y).to(device)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (self.logits[idx], self.labels[idx])

In [ ]:
train_dataloader = BatterySet(train)
test_dataloader = BatterySet(test)

In [ ]:
train_dataloader = DataLoader(train_dataloader, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataloader, batch_size=64)

In [ ]:
train_dataloader.dataset.logits[0]

In [ ]:
train_dataloader.dataset[3600][0]

In [ ]:
for X,y in train_dataloader.dataset:
    print(f"Shape of X [window, features]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
for batch, (x,y) in enumerate(test_dataloader.dataset):
    print(batch,x,y)
    break

# Creating Models <a id="model"></a>

In [ ]:
# # Get cpu or gpu device for training.
# device = torch.device("mps")
# print(f"Using {device} device")

# Define model
class LSTMNetwork(nn.Module):
    
    def __init__(self):
        super(LSTMNetwork, self).__init__()
        
        self.flatten = nn.Flatten()
        self.lstm1 = nn.LSTM(G.num_features, G.batch_size, 3, batch_first = True)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(G.batch_size, G.batch_size // 4),
            nn.ReLU(),
            nn.Linear(G.batch_size // 4, 1)
            )
        self.batch_norm = nn.BatchNorm1d(16)
        
#     def l2_normalize(self, x, dim = 1):
#         "apparently weight decay in the optimize functions does l2 regularization"
#         return nn.functional.normalize(x, p = 2.0 , dim = dim)
    
    def forward(self, x):
#         x = self.flatten(x)
        x_out, x_states = self.lstm1(x)
        x_states = self.batch_norm(x_states[0])
        logits = self.linear_relu_stack(x_out)
        return logits

model = LSTMNetwork().to(device)
print(model)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (x,y) in enumerate(dataloader.dataset):
        #forward
        predict = model(x)
        loss = loss_fn(predict.reshape(y.shape), y)
        
        #backward
        optimizer.zero_grad() #resets the gradient graph, a pytorch shortcoming that is required
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        if batch % 2000 == 0:
            loss, current = loss.item(), batch
            print(f"loss: {loss:>7f}  [{current:5d}/{size:5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = size // G.batch_size
    test_loss, correct = 0,0
    counter = 0
    with torch.no_grad(): #doesnt update parameters (we are testing not training)
        for x,y in dataloader.dataset:
            predict = model(x).reshape(y.shape)
            test_loss += loss_fn(predict, y).item()
            correct+= (predict == y).type(torch.float).sum().item()
            #### will have to change the way correct is computed!!
           
            counter += 1
            if counter % 500 == 0:
                print(f"{counter/14302} done")
            
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {100*correct:>0.1f}%, Avg Loss: {test_loss:>8f}\n")

In [ ]:
loss_fn = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 0.1,
                             weight_decay=1e-5,)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", factor = 0.2, patience = 7, cooldown = 1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 9, gamma=0.5, verbose=False)

In [ ]:
test_loop(test_dataloader, model, loss_fn)

In [ ]:
for epoch in range(G.epochs):
    print(f"Epoch {epoch+1}\n----------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Completed")